In [2]:
#Depedencies
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
api_key = config.coin_api_key
#get the current datetime to add to the file
from datetime import datetime
now = datetime.now()
injest_date = now.strftime("%m/%d/%Y %H:%M:%S")
filedate = now.strftime("%m/%d/%Y")

In [2]:
#API call to CoinMarketCap
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
    'listing_status':'active',
    'limit': 100,
    'sort': 'cmc_rank'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [3]:
#grabs section of dictionary 
cmc_data = data['data']
#puts the dict into a DF, now need to expand nested platform column
cmc_df = pd.DataFrame.from_dict(cmc_data, orient='columns')
#expanded Dataframe
top_coins = pd.concat([cmc_df.drop(['platform'], axis= 1), cmc_df['platform'].apply(pd.Series)], axis= 1 )
#add column file name
top_coins["File_Name"] = "Coin_Ranking_Dim"
#Add column for injested at 
top_coins["injest_datetime"] = injest_date
#turn dataframe to CSV
top_coins = top_coins.to_csv()


 - write DF to parquet format
 `trial_write_to_parquet = test.to_parquet(headers = 'none')`
 - write DF to CSV format
`dfasCSV = test.to_csv()`
 - write DF to string format
`dfAsString = test.to_string(header=False, index=False)`


In [10]:
#this works, needed to send it as text !!
#it works too, had to save it as a CSV
from azure.storage.filedatalake import DataLakeServiceClient
# install the following package 
# pip install azure-storage-file-datalake 
# Get the below details from your storage account
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = "newcontainer"
directory_name = "testingtoo"

service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
        "https", storage_account_name), credential=storage_account_key)
try:
    file_system_client = service_client.create_file_system(file_system=container_name)
    dir_client = file_system_client.get_directory_client(directory_name)
    dir_client.create_directory()
    #set data to appropriate dataframe
    csv_file = top_coins
    file_client = dir_client.create_file("top100coins.csv")
    file_client.append_data(csv_file, 0, len(csv_file))
    file_client.flush_data(len(csv_file))
except  :
    #ResourceAlreadyExists
    file_system_client = service_client.get_file_system_client(file_system=container_name)
    dir_client = file_system_client.get_directory_client(directory_name)
    dir_client.create_directory()
    #set data to appropriate dataframe
    csv_file = top_coins
    file_client = dir_client.create_file("top100coins.csv")
    file_client.append_data(csv_file, 0, len(csv_file))
    file_client.flush_data(len(csv_file))


### -----------------------------------------------------------------------------------------------------------------------------------------